In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.action_chains import ActionChains
import csv
import scrapy
import time
import re
import pandas as pd

In [19]:
#you will need to put your own path to the firefox binary here
#binary = FirefoxBinary()
legistar = 'https://milwaukee.legistar.com/Legislation.aspx'

def visit_site(legistar):
    mkeDriver = webdriver.Firefox(executable_path='/Users/ayaspan/Documents/Personal/voting_records_mke/geckodriver')
    mkeDriver.get(legistar)
    return mkeDriver


In [375]:
mkeDriver2 = visit_site(legistar)

In [385]:
test = get_next_page(mkeDriver2)

[]


In [3]:
def select_all_years(mkeDriver):
    years = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_lstYears_Input")
    years.send_keys('All Years')
    years.submit()
    #return years

In [4]:
#makes it so that you have max results

def get_max_results(mkeDriver):
    step1 = mkeDriver.find_elements_by_id("ctl00_ContentPlaceHolder1_menuMain")[0]
    action = ActionChains(mkeDriver)
    action.click(step1.find_elements_by_class_name("rmItem ")[1])
    #action.click(step1.find_elements_by_class_name("rmItem ")[1].find_element_by_css_selector("a"))
    action.perform()
    time.sleep(1)
    step2 = step1.find_elements_by_class_name("rmItem ")[1].parent
    #print(step2.text)
    action = ActionChains(mkeDriver)
    # action_test2 = ActionChains(testDriver)
    action.move_to_element(step2.find_element_by_class_name("rmSlide").find_element_by_class_name("rmLast"))
    action.click(step2.find_element_by_class_name("rmSlide").find_element_by_class_name("rmLast"))
    action.perform()
    #return mkeDriver

In [5]:
# re.search(r'(?<=of\s)[0-9]+',mkeDriver.find_elements_by_class_name("rgPagerCell")[0]\
#             .find_element_by_class_name("rgInfoPart").text)[0]

In [6]:
#need to consider the recursion involved in looping through pages OR use above

In [7]:
#locats page link 
def get_next_page(mkeDriver):
    pgs_drop = mkeDriver.find_elements_by_class_name("rgPagerCell")
    #print(pgs_drop)
    try:
        return pgs_drop[0].find_element_by_xpath("//a[@class='rgCurrentPage']/following-sibling::a")
    except:
        return None

def go_to_next_page(next_page):
    next_page.click()

In [8]:
def select_ordinance(mkeDriver):
    type_ordinance = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_lstTypeBasic_Arrow").click()
    legislation_type = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_lstTypeBasic_DropDown")\
                .find_elements_by_class_name("rcbItem")
    print(legislation_type[14].text)
    actions = ActionChains(mkeDriver)
    actions.move_to_element(legislation_type[14])
    actions.click(legislation_type[14])
    actions.perform()
    #choose_ordinance.perform()#.double_click()
#type_legislation.s
#type_legislation.submit()

In [9]:
# legislation_type = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_lstTypeBasic_DropDown")\
#                 .find_elements_by_class_name("rcbItem")
# hover_ord = ActionChains(mkeDriver).move_to_element(legislation_type[14])
# hover_ord.perform().send_keys("Ordinance").double_click()

In [10]:
def start_search(mkeDriver):
    s_button = mkeDriver.find_element_by_id("visibleSearchButton")
    s_button.click()
    #return mkeDriver

In [65]:
def get_records_table(mkeDriver):
    records_table = mkeDriver.find_element_by_id("ctl00_ContentPlaceHolder1_gridMain_ctl00")
    r_table_body = records_table.find_elements_by_class_name("rgRow")
    return r_table_body

In [212]:
# for i in records_table.find_elements_by_css_selector('a'):
#     if 'https' in i.get_attribute('href'):
#         print(i.get_attribute('href'))

In [336]:
#r_table_body = records_table.find_element_by_css_selector("tbody")

In [ ]:
#can change # of records per page :)

In [81]:
#if status passed
import time

def get_meta_data(legistar):
    count=0
    leg_dict = {}
    col_names = [0,'type','status','created_date','final_action','title']
    
    #CALL OTHER REQUISITE FUNCTIONS FIRST
    mkeDriver = visit_site(legistar)
    #time.sleep(5)
    select_all_years(mkeDriver)
    time.sleep(2)
    select_ordinance(mkeDriver)
    start_search(mkeDriver)
    time.sleep(2)
    get_max_results(mkeDriver)
    
    while True:
        time.sleep(2)
        r_table_body = get_records_table(mkeDriver)
        leg_dict[count]= {i.find_element_by_css_selector('td').find_element_by_css_selector('a').text:{
                          'link': i.find_element_by_id('ctl00_ContentPlaceHolder1_gridMain_ctl00_ctl04_hypFile').get_attribute('href'),
                           'other_info':[j.text for j in i.find_elements_by_css_selector('td')]} for i in r_table_body}
        #works
        leg_dict[count] = [i.text for i in r_table_body]
        
        #print(r_table_body.text)
        #print(r_table_body.find_elements_by_class_name('rgRow')[0].text)
#         for row in r_table_body.find_elements_by_class_name('rgRow'):
#             print(row.get_attribute('id'),'id')
#             file_num = row.find_element_by_css_selector('td').find_element_by_css_selector('a').text
#             leg_dict[file_num] = {}
#             link = row.find_element_by_id('ctl00_ContentPlaceHolder1_gridMain_ctl00_ctl04_hypFile').get_attribute('href')
#             leg_dict[file_num]['file_link'] = link
#             print(file_num,'file_num and ', link,' link')

#             for d,col in zip(row.find_elements_by_css_selector('td'),col_names):
#                 if not col:
#                     continue
#                 #if leg_dict[file_num]['status'] in ('Passed',"Mayor's Office",'Dead','Placed On File'): use this later for going through links
#                 leg_dict[file_num][col] = d.text
# #                 leg_dict[file_num]['status'] = d.text
# #                 leg_dict[file_num]['created_date'] = d.text
# #                 leg_dict[file_num]['final_action'] = d.text
#                 leg_dict[file_num]['title'] = d.text
        time.sleep(.5)
        count+=1
        
        next_page = get_next_page(mkeDriver)
        
        if not next_page:
            return leg_dict
        else:
            next_page.click()
        
        
        count+=1
    

In [82]:
#leg_dict

In [84]:
meta_legis_dict = get_meta_data(legistar)

Ordinance


NoSuchElementException: Message: Unable to locate element: [id="ctl00_ContentPlaceHolder1_gridMain_ctl00_ctl04_hypFile"]


In [77]:
#meta_legis_dict[2] #worked

In [83]:
meta_legis_dict2 = get_meta_data(legistar)

Ordinance


NoSuchElementException: Message: Unable to locate element: [id="ctl00_ContentPlaceHolder1_gridMain_ctl00_ctl04_hypFile"]


In [ ]:
meta_leg_df = pd.DataFrame.from_dict(meta_legis_dict)

In [ ]:
def objectIdentificationUsingWaits(self, maxTimeOut, locatorProperties, locatorType = "xpath"):
        element = None
        try:
            WebDriverWait(self.driver, maxTimeOut, ignored_exceptions=[StaleElementReferenceException]).until(
                EC.presence_of_element_located((self.getLocatorType(locatorType), locatorProperties)))
            element = self.driver.find_element(locatorType, locatorProperties)
        except:
            print("Exception occurred during object identification.")
        return element

In [662]:
from selenium.common.exceptions import StaleElementReferenceException

test = visit_site(legistar)
#time.sleep(5)
select_all_years(test)
time.sleep(5)
select_ordinance(test)
start_search(test)
time.sleep(3)
get_max_results(test)
time.sleep(.5)



col_names = [0,'type','status','created_date','final_action','title']
leg_dict = {}
r_table_body = get_records_table(test)
print(r_table_body.text)
print(r_table_body.find_elements_by_class_name('rgRow'))
#print(r_table_body.find_elements_by_class_name('rgRow')[0].text)
for row in r_table_body.find_elements_by_class_name('rgRow'):
    print(row.get_attribute('id'),'id')
    file_num = row.find_element_by_css_selector('td').find_element_by_css_selector('a').text
    leg_dict[file_num] = {}
    try:
        link = row.find_element_by_id('ctl00_ContentPlaceHolder1_gridMain_ctl00_ctl04_hypFile').get_attribute('href')
    except:
        link='no link'
    leg_dict[file_num]['file_link'] = link
    print(file_num,'file_num and ', link,' link')
    for d,col in zip(row.find_elements_by_css_selector('td'),col_names):
                if not col:
                    continue
                
                leg_dict[file_num][col] = d.text

Ordinance
181536 Ordinance In Committee 1/14/2019   An ordinance establishing a Site Plan Review Overlay Zone for the property located at 6000 South 6th Street, on the east side of South 6th Street, north of West College Avenue, in the 13th Aldermanic District.
[]
